In [1]:
review_file = 'test_set_all.txt'
import csv
x_train_full = list()
with open(review_file, encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile)
    for row in reader:
        x_train_full.append(row[0])
        
import random
from random import seed

seed(2)

x_train = random.sample(x_train_full, 1000)

In [2]:
x_train

['Echo Auto  Hands free Alexa in your car with your phone\tI’m 70 years old and not the best at techie things but once this was synced with all I have to do is ask, done.It’s sort of like the genie in a bottle but you have more then three wishes.  I love it I never have to take my eyes off the road\t4\tJuly 15, 2021\t1\t\t\t\t\t\t\t\t',
 'Echo Dot\t\t5\tAugust 14, 2022\t1\t\t\t\t\t\t\t\t',
 '                \t\t\t\t\t\t\t\t',
 'Echo 4th Gen\tI had to buy a new, more expensive, larger, uglier Alexa device because the old one could not connect devices on a modern Wi-Fi router with 5G. She is not capable of answering almost any question, including her own model or specifications. And after a dozen tries, she still could not connect to my device. Alexa used to be far simpler, far less expensive, and work far better.Is it time to convert to Google smart home, or is that just as bad?Addition: After posting this review, Amazon Customer Service called me and was able to assist me in connecting

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

en_stopwords = stopwords.words('english')

tf_vectorizer = CountVectorizer(stop_words=en_stopwords, max_df=0.5, min_df=5,max_features = 7000, ngram_range=(1,2))
tf = tf_vectorizer.fit_transform(x_train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alice\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
tf[0]

<1x1729 sparse matrix of type '<class 'numpy.int64'>'
	with 30 stored elements in Compressed Sparse Row format>

In [5]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 8

lda = LatentDirichletAllocation(n_components=n_components, max_iter=20,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=1)
lda.fit(tf)


iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


LatentDirichletAllocation(max_iter=20, n_components=8, n_jobs=-1, verbose=1)

In [6]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print()
        message = f'Topic {topic_idx}: '
        message += ', '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [7]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topic 0: gen, 4th, 4th gen, echo 4th, alexa, amazon, device, music, 2021, 2022, one, new, get, ask, like, play, playing, volume, say, commands

Topic 1: music, alexa, love, use, easy, set, sound, also, time, lights, really, 2021, smart, one, great, alarm, house, even, dots, things

Topic 2: dot, echo dot, 2022, august, de, september, que, el, la, en, se, es, alexa, love, august 2022, 17, july, recognition, muy, october

Topic 3: sound, quality, studio, dot, alexa, echo studio, better, music, one, echo dot, good, sound quality, like, 2021, speaker, amazon, would, 2020, home, get

Topic 4: show, echo show, 10, show 10, screen, 2021, video, 2022, like, one, work, use, camera, march, get, device, kitchen, feature, amazon, great

Topic 5: tv, amazon, fire tv, fire, could, echo dot, dot, remote, media, app, could loaded, media could, loaded, blink, speaker, monthly, bluetooth, contacts, bluetooth speaker, would

Topic 6: dot, echo dot, one, alexa, well, 2022, set, like, october, things, int

In [8]:
import pyLDAvis
import pyLDAvis.sklearn


In [9]:
pyLDAvis.enable_notebook()

In [10]:
pyLDAvis.sklearn.prepare(lda,tf,tf_vectorizer)

C:\Users\Alice\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.043074 -0.058721       1        1  30.198827
7     -0.116512 -0.038199       2        1  23.181747
0     -0.082008  0.012439       3        1  13.751919
4     -0.068253  0.235597       4        1  13.682207
1     -0.048497 -0.113339       5        1   7.145506
2      0.316764  0.017726       6        1   5.550938
6      0.051405 -0.025097       7        1   4.143022
5     -0.009826 -0.030407       8        1   2.345834, topic_info=           Term        Freq       Total Category  logprob  loglift
1352       show  438.000000  438.000000  Default  30.0000  30.0000
475         dot  423.000000  423.000000  Default  29.0000  29.0000
516    echo dot  364.000000  364.000000  Default  28.0000  28.0000
521   echo show  341.000000  341.000000  Default  27.0000  27.0000
51         2022  361.000000  361.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
1352       show    5.451123  438.074908   Topic8  -5.1914  -0.6340
870      listen    4.353241   64.209677   Topic8  -5.4163   1.0613
1632      voice    4.643453  172.972007   Topic8  -5.3518   0.1349
521   echo show    4.691112  341.433844   Topic8  -5.3416  -0.5350
1602        use    4.643718  257.283114   Topic8  -5.3517  -0.2621

[514 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         2  0.052596     10
0         3  0.038252     10
0         4  0.812847     10
0         6  0.033470     10
0         7  0.062159     10
...     ...       ...    ...
1709      4  0.099212  would
1709      5  0.060390  would
1709      7  0.025881  would
1709      8  0.030195  would
1720      6  0.951398     ya

[1087 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 1, 5, 2, 3, 7, 6])

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 3

lda = LatentDirichletAllocation(n_components=n_components, max_iter=20,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=1)
lda.fit(tf)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


LatentDirichletAllocation(max_iter=20, n_components=3, n_jobs=-1, verbose=1)

In [12]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print()
        message = f'Topic {topic_idx}: '
        message += ', '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [13]:
n_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topic 0: dot, echo dot, 2022, gen, sound, 4th, 4th gen, echo 4th, studio, echo studio

Topic 1: alexa, phone, car, auto, music, echo auto, sound, free, like, better

Topic 2: show, echo show, 10, amazon, one, alexa, show 10, 2021, device, screen



In [14]:
n_top_words = 5
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topic 0: dot, echo dot, 2022, gen, sound

Topic 1: alexa, phone, car, auto, music

Topic 2: show, echo show, 10, amazon, one



Prendendo solo colonna 'verified_reviews'

In [15]:
import pandas as pd
df=pd.read_csv('test_set_all.csv',sep='\t', usecols=["verified_reviews"])

In [16]:
print(df)

                                        verified_reviews
0      This is an honest review and comparison of the...
1      A friend of mine recently gave me an echo. 2nd...
2      The new Echo 4th Gen is a big change in aesthe...
3      I got the Echo out of curiosity after hearing ...
4      I already had an echo, I added a sub about 3 m...
...                                                  ...
32071  Great sound and Alexa integration. Only issue ...
32072  Does not get that loud.  Have to go to 3/4 str...
32073  As far as sound quality this is a great purcha...
32074  Right now, it has a long lead time.  I ordered...
32075  I bought this hoping for clear sound, but its ...

[32076 rows x 1 columns]


In [17]:
pip install aspose-cells

Note: you may need to restart the kernel to use updated packages.


In [18]:
df.to_csv('df.csv')

In [19]:
file = 'df.txt'
import csv
df_train_full = list()
with open(file, encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile)
    for row in reader:
        df_train_full.append(row[0])
        
import random
from random import seed

seed(2)

df_train = random.sample(df_train_full, 1000)

In [20]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

en_stopwords = stopwords.words('english')

tfd_vectorizer = CountVectorizer(stop_words=en_stopwords, max_df=0.5, min_df=5,max_features = 7000, ngram_range=(1,2))
tfd = tfd_vectorizer.fit_transform(df_train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alice\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
tfd[0]

<1x1271 sparse matrix of type '<class 'numpy.int64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [22]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 5

lda = LatentDirichletAllocation(n_components=n_components, max_iter=20,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=1)
lda.fit(tfd)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


LatentDirichletAllocation(max_iter=20, n_components=5, n_jobs=-1, verbose=1)

In [23]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print()
        message = f'Topic {topic_idx}: '
        message += ', '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [24]:
n_top_words = 10
tfd_feature_names = tfd_vectorizer.get_feature_names()
print_top_words(lda, tfd_feature_names, n_top_words)


Topic 0: alexa, car, phone, music, echo, auto, get, use, bluetooth, echo auto

Topic 1: echo, show, sound, alexa, one, use, room, echo show, devices, speaker

Topic 2: sound, quality, echo, music, one, dot, sound quality, alexa, better, gen

Topic 3: alexa, time, love, would, echo, like, ask, work, phone, app

Topic 4: amazon, device, echo, one, show, could, screen, echo show, support, new



In [25]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [26]:
pyLDAvis.sklearn.prepare(lda,tfd,tfd_vectorizer)

C:\Users\Alice\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.048955  0.091656       1        1  28.163831
1     -0.010062 -0.106597       2        1  24.233032
2     -0.087068 -0.094773       3        1  20.318496
3     -0.056209  0.104572       4        1  16.663464
4      0.202295  0.005144       5        1  10.621176, topic_info=         Term        Freq       Total Category  logprob  loglift
1010    sound  277.000000  277.000000  Default  30.0000  30.0000
172       car  157.000000  157.000000  Default  29.0000  29.0000
65     amazon  226.000000  226.000000  Default  28.0000  28.0000
275    device  240.000000  240.000000  Default  27.0000  27.0000
834   quality  141.000000  141.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1255    would   19.981038  188.810616   Topic5  -4.9598  -0.0036
0          10   14.427877   47.373737   Topic5  -5.2855   1.0534
820   product   15.375693   87.007365   Topic5  -5.2218   0.5091
133    better   15.319219  142.301089   Topic5  -5.2255   0.0135
38      alexa   14.620534  515.709869   Topic5  -5.2722  -1.3208

[356 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         2  0.506610     10
0         4  0.189979     10
0         5  0.295522     10
8         1  0.041743    3rd
8         3  0.918339    3rd
...     ...       ...    ...
1255      2  0.227741  would
1255      3  0.127111  would
1255      4  0.275408  would
1255      5  0.105926  would
1267      3  0.913848   yell

[664 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

In [27]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 3

lda = LatentDirichletAllocation(n_components=n_components, max_iter=20,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=1)
lda.fit(tfd)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


LatentDirichletAllocation(max_iter=20, n_components=3, n_jobs=-1, verbose=1)

In [28]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print()
        message = f'Topic {topic_idx}: '
        message += ', '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [29]:
n_top_words = 10
tfd_feature_names = tfd_vectorizer.get_feature_names()
print_top_words(lda, tfd_feature_names, n_top_words)


Topic 0: sound, echo, quality, better, alexa, music, sound quality, gen, good, speaker

Topic 1: alexa, phone, echo, car, device, music, would, work, auto, get

Topic 2: echo, show, alexa, amazon, like, one, screen, echo show, music, use



In [30]:
n_top_words = 6
tfd_feature_names = tfd_vectorizer.get_feature_names()
print_top_words(lda, tfd_feature_names, n_top_words)


Topic 0: sound, echo, quality, better, alexa, music

Topic 1: alexa, phone, echo, car, device, music

Topic 2: echo, show, alexa, amazon, like, one



In [31]:
import pyLDAvis
import pyLDAvis.sklearn

In [32]:
pyLDAvis.enable_notebook()

In [33]:
pyLDAvis.sklearn.prepare(lda,tfd,tfd_vectorizer)

C:\Users\Alice\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.064238  0.094231       1        1  39.084116
2     -0.081038 -0.087236       2        1  32.488064
0      0.145276 -0.006995       3        1  28.427820, topic_info=         Term        Freq       Total Category  logprob  loglift
1010    sound  273.000000  273.000000  Default  30.0000  30.0000
966      show  165.000000  165.000000  Default  29.0000  29.0000
834   quality  139.000000  139.000000  Default  28.0000  28.0000
172       car  157.000000  157.000000  Default  27.0000  27.0000
765     phone  199.000000  199.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
150    bought   34.635779   68.384780   Topic3  -5.3943   0.5775
573      like   53.189349  232.108068   Topic3  -4.9653  -0.2155
1255    would   48.752951  189.415834   Topic3  -5.0524  -0.0994
443       get   42.595094  204.745967   Topic3  -5.1874  -0.3122
1169      use   41.314850  198.888167   Topic3  -5.2179  -0.3137

[226 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.104461     10
0         2  0.877469     10
8         1  0.063611    3rd
8         2  0.127223    3rd
8         3  0.826946    3rd
...     ...       ...    ...
1245      2  0.181614  works
1245      3  0.124262  works
1255      1  0.575453  would
1255      2  0.168940  would
1255      3  0.258690  would

[318 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [34]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 3

lda = LatentDirichletAllocation(n_components=n_components, max_iter=20,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=1)
lda.fit(tfd)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


LatentDirichletAllocation(max_iter=20, n_components=3, n_jobs=-1, verbose=1)

In [35]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print()
        message = f'Topic {topic_idx}: '
        message += ', '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [36]:
n_top_words = 6
tfd_feature_names = tfd_vectorizer.get_feature_names()
print_top_words(lda, tfd_feature_names, n_top_words)


Topic 0: alexa, phone, echo, car, device, music

Topic 1: sound, echo, music, quality, alexa, one

Topic 2: show, echo, alexa, like, screen, one



In [37]:
pyLDAvis.enable_notebook()

In [38]:
pyLDAvis.sklearn.prepare(lda,tfd,tfd_vectorizer)

C:\Users\Alice\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.109174  0.063454       1        1  37.268308
0      0.002913 -0.122086       2        1  36.260591
2     -0.112087  0.058632       3        1  26.471101, topic_info=         Term        Freq       Total Category  logprob  loglift
966      show  164.000000  164.000000  Default  30.0000  30.0000
1010    sound  277.000000  277.000000  Default  29.0000  29.0000
765     phone  198.000000  198.000000  Default  28.0000  28.0000
172       car  156.000000  156.000000  Default  27.0000  27.0000
834   quality  141.000000  141.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
275    device   49.635601  241.861889   Topic3  -4.9631  -0.2545
922       say   36.256655   96.337965   Topic3  -5.2772   0.3519
300       dot   36.851237  119.148498   Topic3  -5.2609   0.1556
499      home   36.552499  115.463614   Topic3  -5.2691   0.1789
952       set   32.926506   83.906541   Topic3  -5.3735   0.3937

[222 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         1  0.021058           10
0         2  0.126347           10
0         3  0.863371           10
11        1  0.963093          4th
12        1  0.947151      4th gen
...     ...       ...          ...
1247      2  0.933392  works great
1251      1  0.942171        worse
1255      1  0.496649        would
1255      2  0.390979        would
1255      3  0.110954        would

[309 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])